In [ ]:

# Step 1: Install the netCDF4 library
#!pip install netCDF4

# Step 2: Import the netCDF4 library
import netCDF4 as nc
from netCDF4 import Dataset

# Step 3: Specify the path to your NetCDF file
file_path = 'merged_daily_sea_ice.nc'

# Step 4: Open and read the NetCDF file
fhadj = Dataset(file_path, mode='r')

# Step 5: Explore the file
# Print out the variables in the file
print("Variables in the file:")
print(fhadj.variables.keys())



# Close the dataset when done


y = fhadj.variables['y'][:]  # Latitude
x = fhadj.variables['x'][:]  # Longitude``
SIC = fhadj.variables['SIC'][:]  # Sea Ice Concentration
year = fhadj.variables['year'][:]  # Year
month = fhadj.variables['month'][:]  # Month
day = fhadj.variables['day'][:]  # Day



import numpy as np

# Assuming SIC is a 3D NumPy array with dimensions (time, latitude, longitude)
# You can adjust the axis parameter based on your data structure

# Filter along the time dimension (axis=0)
filtered_SIC = np.where(SIC <= 1, SIC, 0)

# Now, filtered_SIC contains only the values in SIC that are less than or equal to 1 along the time dimension
print(np.min(filtered_SIC))
print(np.max(filtered_SIC))





print(list(y))
print(list(x))

In [ ]:
# xyList = []
# yList = []

# for x1 in x:
#     for y1 in y:
#         xyList.append((x1,y1))





In [ ]:
import os

os.remove('xr_data.nc')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyproj import CRS, Transformer

# Define the projection for your data (e.g., EPSG 3411 for Polar Stereographic)
input_projection = CRS.from_string('epsg:3412')


# Create a grid of coordinates
xx, yy = np.meshgrid(x, y)

# Create a transformer for the coordinate conversion
transformer = Transformer.from_crs(input_projection, CRS.from_string('epsg:4326'))



# Convert grid coordinates to latitude and longitude
lon1, lat1 = transformer.transform(xx, yy)


print(xx.shape)

In [ ]:
print(np.min(lat1))


print(np.max(lat1))


print(np.min(lon1))

print(np.max(lon1))

In [ ]:
import numpy as np
import datetime
from netCDF4 import Dataset




year = year.astype(int)
month = month.astype(int)
day = day.astype(int)


date_var = []
for i in range(len(year)):
    sum = (year[i]*100+month[i])*100+day[i]
    date_var.append(sum)
    





# Create a NetCDF file for writing
output_file = 'xr_data.nc'
nc_dataset = Dataset(output_file, 'w', format='NETCDF4')

# Define dimensions in the NetCDF file
nc_dataset.createDimension('time', len(date_var))


nc_dataset.createDimension('y', 332)
nc_dataset.createDimension('x', 316)


# Create variables in the NetCDF file
time_var = nc_dataset.createVariable('time', 'i8', ('time',))
lat_var = nc_dataset.createVariable('lat', 'f4', ('y', 'x'))
lon_var = nc_dataset.createVariable('lon', 'f4', ('y', 'x'))
sic_var = nc_dataset.createVariable('SIC', 'f4', ('time', 'y', 'x'))

# Assign data to variables
time_var[:] = date_var
lat_var[:,:] = lon1
lon_var[:,:] = lat1
sic_var[:, :, :] = filtered_SIC



# Close the NetCDF file
nc_dataset.close()


In [ ]:
# import os

# os.remove('xr_data.nc')

In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import xesmf as xe

ds = xr.open_dataset("xr_data.nc")
ds = ds.set_coords(['lat', 'lon'])

print(ds)


In [ ]:

# dr = ds["SIC"]


# plt.figure(figsize=(12, 4))
# ax = plt.axes(projection=ccrs.PlateCarree())
# dr[880].plot.pcolormesh(ax=ax, x="lon", y="lat")
# ax.coastlines()



In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Assuming you have loaded your dataset (ds)
# ds = xr.open_dataset("your_dataset.nc")

# Extract the SIC data variable
dr = ds["SIC"]

# Select a specific time index (e.g., 880)
time_index = 880
data_slice = dr[time_index]

# Define the Antarctic region bounds
#antarctic_extent = [-180, 180, -90, 90]  # Adjust the bounds as needed

# Plot the Antarctic region
plt.figure(figsize=(12, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
pcm = data_slice.plot.pcolormesh(ax=ax, x="lon", y="lat", cmap="viridis")

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', linestyle='--')

# Set the extent to the Antarctic region
#ax.set_extent(antarctic_extent, crs=ccrs.PlateCarree())

# Add labels to x and y axes
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Add colorbar
#cbar = plt.colorbar(pcm, orientation='vertical', pad=0.05, label='Sea Ice Concentration')

plt.show()




In [ ]:

dr = ds["SIC"]
print(dr.shape)

plt.figure(figsize=(12, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
dr[5000].plot.pcolormesh(ax=ax, x="lon", y="lat")
ax.coastlines()

ds_out = xe.util.grid_global(0.25, 0.25)


regridder = xe.Regridder(ds, ds_out, "bilinear")
dr_out = regridder(dr)



In [ ]:
plt.figure(figsize=(12, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
dr_out[67].plot.pcolormesh(ax=ax, x="lon", y="lat")
ax.coastlines()

In [ ]:
dr_out['lat']

In [ ]:

# Create a NetCDF file for writing
output_file = 'data_25_degree.nc'
nc_dataset = Dataset(output_file, 'w', format='NETCDF4')

# Define dimensions in the NetCDF file
nc_dataset.createDimension('time', len(date_var))


nc_dataset.createDimension('y', 720)
nc_dataset.createDimension('x', 1440)


# Create variables in the NetCDF file
time_var = nc_dataset.createVariable('time', 'i8', ('time',))
lon_var = nc_dataset.createVariable('lon', 'f4', ('y', 'x'))
lat_var = nc_dataset.createVariable('lat', 'f4', ('y', 'x'))



sic_var = nc_dataset.createVariable('SIC', 'f4', ('time', 'y', 'x'))

# Assign data to variables
time_var[:] = date_var
lat_var[:,:] = dr_out['lat']
lon_var[:,:] = dr_out['lon']
sic_var[:, :, :] = dr_out[:]



# Close the NetCDF file
nc_dataset.close()

In [ ]:
file_path = 'data_25_degree.nc'

%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import xesmf as xe

ds = xr.open_dataset(file_path)
ds = ds.set_coords(['lat', 'lon'])

print(ds)


print('**********************************')
print("Variables in the file:")
print(ds.variables.keys())

dr_out = ds['SIC'][:]

plt.figure(figsize=(12, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
dr_out[3000].plot.pcolormesh(ax=ax, x="lon", y="lat")
ax.coastlines()


ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', linestyle='--')

# Set the extent to the Antarctic region
#ax.set_extent(antarctic_extent, crs=ccrs.PlateCarree())

# Add labels to x and y axes
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')